In [1]:
!pip install pennylane

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 957 kB 5.1 MB/s 
     |████████████████████████████████| 9.3 MB 26.3 MB/s 
     |████████████████████████████████| 1.6 MB 35.8 MB/s 
     |████████████████████████████████| 108 kB 48.9 MB/s 


In [2]:
import pandas as pd
import numpy as np
from pennylane import numpy as np
from sklearn.preprocessing import normalize
from sklearn.preprocessing import StandardScaler

import pennylane as qml
from pennylane.templates.embeddings import AngleEmbedding, AmplitudeEmbedding
from pennylane.optimize import AdamOptimizer
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler


import time
start = time.time()

In [3]:
df = pd.read_csv('/content/ClaMP_Integrated-5184.csv', sep=',')

In [4]:
df.drop(['packer_type'],axis=1,inplace=True)

In [5]:
from sklearn.utils import shuffle
df = shuffle(df)

In [6]:
# Separate class
class_0 = df[df['class'] == 0]
class_1 = df[df['class'] == 1]# print the shape of the class
print('class 0:', class_0.shape)
print('class 1:', class_1.shape)

class 0: (2488, 69)
class 1: (2722, 69)


In [7]:
df = df.reset_index(level=None, drop=False, inplace=False, col_level=0, col_fill='')

In [8]:
df.drop(['index'],axis=1,inplace=True)

In [9]:
df

,e_cblp,e_cp,e_cparhdr,e_maxalloc,e_sp,e_lfanew,NumberOfSections,CreationYear,FH_char0,FH_char1,...,LoaderFlags,sus_sections,non_sus_sections,packer,E_text,E_data,filesize,E_file,fileinfo,class
0,144,3,4,65535,184,232,5,1,0,1,...,1,1,4,0,6.376328,1.754106,80896,6.490903,1,0
1,144,3,4,65535,184,184,4,1,1,1,...,1,0,4,0,7.193251,7.794249,982016,7.496236,0,1
2,144,3,4,65535,184,232,6,1,0,1,...,1,2,4,0,6.228274,0.000000,92160,7.635519,1,1
3,144,3,4,65535,184,128,5,1,1,1,...,1,0,5,0,1.971186,5.077625,487424,7.757292,0,1
4,144,3,4,65535,184,128,5,1,1,1,...,1,0,5,0,6.379094,7.035748,332288,7.799451,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5205,144,3,4,65535,184,240,5,1,1,1,...,1,1,4,0,4.651197,7.996547,98304,7.417740,0,1
5206,144,3,4,65535,184,280,4,1,0,1,...,1,1,3,0,6.340451,7.295707,4794528,6.529445,1,0
5207,144,3,4,65535,184,184,1,1,0,1,...,1,0,1,0,0.000000,0.000000,93696,3.640529,1,0
5208,144,3,4,65535,184,224,5,1,0,1,...,1,1,4,0,6.251547,3.507408,165376,6.091977,1,0


In [10]:
df_5percent=df[0:4428]

In [11]:
df_5percent

,e_cblp,e_cp,e_cparhdr,e_maxalloc,e_sp,e_lfanew,NumberOfSections,CreationYear,FH_char0,FH_char1,...,LoaderFlags,sus_sections,non_sus_sections,packer,E_text,E_data,filesize,E_file,fileinfo,class
0,144,3,4,65535,184,232,5,1,0,1,...,1,1,4,0,6.376328,1.754106,80896,6.490903,1,0
1,144,3,4,65535,184,184,4,1,1,1,...,1,0,4,0,7.193251,7.794249,982016,7.496236,0,1
2,144,3,4,65535,184,232,6,1,0,1,...,1,2,4,0,6.228274,0.000000,92160,7.635519,1,1
3,144,3,4,65535,184,128,5,1,1,1,...,1,0,5,0,1.971186,5.077625,487424,7.757292,0,1
4,144,3,4,65535,184,128,5,1,1,1,...,1,0,5,0,6.379094,7.035748,332288,7.799451,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4423,144,3,4,65535,184,208,3,1,1,1,...,1,2,1,1,0.000000,0.000000,187392,7.925639,1,1
4424,144,3,4,65535,184,272,6,1,0,1,...,1,2,4,0,6.331827,3.828319,230000,6.268068,1,0
4425,144,3,4,65535,184,200,3,1,1,1,...,1,2,1,1,0.000000,0.000000,53620,7.845928,0,1
4426,144,3,4,65535,184,216,4,1,0,1,...,1,1,3,0,6.383862,0.945373,2613248,5.931275,1,0


In [12]:
# Separate class
class_0 = df_5percent[df_5percent['class'] == 0]
class_1 = df_5percent[df_5percent['class'] == 1]# print the shape of the class
print('class 0:', class_0.shape)
print('class 1:', class_1.shape)

class 0: (2131, 69)
class 1: (2297, 69)


In [13]:
# Read out CSV and sets/samples creation

df_5percent = df_5percent.astype(float)
train,test = train_test_split(df_5percent, test_size=0.30, random_state=2)
train_set = train
test_set = test
train_set = train_set.sample(160)
test_set = test_set.sample(40)
np.random.seed(42)

In [14]:
df_5percent.info

<bound method DataFrame.info of       e_cblp  e_cp  e_cparhdr  e_maxalloc   e_sp  e_lfanew  NumberOfSections  \
0      144.0   3.0        4.0     65535.0  184.0     232.0               5.0   
1      144.0   3.0        4.0     65535.0  184.0     184.0               4.0   
2      144.0   3.0        4.0     65535.0  184.0     232.0               6.0   
3      144.0   3.0        4.0     65535.0  184.0     128.0               5.0   
4      144.0   3.0        4.0     65535.0  184.0     128.0               5.0   
...      ...   ...        ...         ...    ...       ...               ...   
4423   144.0   3.0        4.0     65535.0  184.0     208.0               3.0   
4424   144.0   3.0        4.0     65535.0  184.0     272.0               6.0   
4425   144.0   3.0        4.0     65535.0  184.0     200.0               3.0   
4426   144.0   3.0        4.0     65535.0  184.0     216.0               4.0   
4427   144.0   3.0        4.0     65535.0  184.0     184.0               1.0   

      C

In [15]:
df_5percent.describe()

,e_cblp,e_cp,e_cparhdr,e_maxalloc,e_sp,e_lfanew,NumberOfSections,CreationYear,FH_char0,FH_char1,...,LoaderFlags,sus_sections,non_sus_sections,packer,E_text,E_data,filesize,E_file,fileinfo,class
count,4428.000000,4428.000000,4428.000000,4428.000000,4428.000000,4428.000000,4428.000000,4428.000000,4428.000000,4428.0,...,4428.000000,4428.000000,4428.000000,4428.000000,4428.000000,4428.000000,4.428000e+03,4428.000000,4428.000000,4428.000000
mean,154.975610,11.995935,9.773713,65141.439928,205.980578,222.785908,4.670732,0.990063,0.359756,1.0,...,0.999097,1.378726,3.292005,0.155601,4.908445,2.492374,7.903893e+05,6.350252,0.547200,0.518744
std,668.664794,426.008566,273.001584,4966.112255,1050.640716,49.348722,1.971099,0.099198,0.479983,0.0,...,0.030045,1.672934,1.147319,0.362517,2.534722,2.674897,5.475298e+06,1.122550,0.497823,0.499705
min,0.000000,0.000000,0.000000,0.000000,0.000000,12.000000,1.000000,0.000000,0.000000,1.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.536000e+03,0.939626,0.000000,0.000000
25%,144.000000,3.000000,4.000000,65535.000000,184.000000,208.000000,4.000000,1.000000,0.000000,1.0,...,1.000000,1.000000,3.000000,0.000000,3.839223,0.000000,6.042800e+04,5.678277,0.000000,0.000000
50%,144.000000,3.000000,4.000000,65535.000000,184.000000,232.000000,5.000000,1.000000,0.000000,1.0,...,1.000000,1.000000,4.000000,0.000000,6.127913,1.500657,1.205760e+05,6.375392,1.000000,1.000000
75%,144.000000,3.000000,4.000000,65535.000000,184.000000,248.000000,5.000000,1.000000,1.000000,1.0,...,1.000000,2.000000,4.000000,0.000000,6.497885,4.650353,3.010740e+05,7.287788,1.000000,1.000000
max,37008.000000,20050.000000,12851.000000,65535.000000,65534.000000,648.000000,34.000000,1.000000,1.000000,1.0,...,1.000000,31.000000,8.000000,1.000000,7.999859,7.999620,1.657081e+08,7.999997,1.000000,1.000000


In [16]:
# Separation of labels

x_train = train_set
y_train = train_set[['class']]

x_test = test_set
y_test = test_set[['class']]

In [17]:
# Reduce dimensions using PCA to fit the dimensions with the qubits

n_dim = 2
pca = PCA(n_components=n_dim)
pca.fit(x_train)

x_train = pca.transform(x_train)

pca.fit(x_test)
x_test = pca.transform(x_test)

In [18]:
# Normalize

std_scale = StandardScaler().fit(x_train)
data = std_scale.transform(x_train)

std_scale = StandardScaler().fit(x_test)
x_test = std_scale.transform(x_test)

In [19]:
# Review the balance of the target variable in train

y_train.value_counts(normalize=True)*100

class
1.0      54.375
0.0      45.625
dtype: float64

In [20]:
# Review the balance of the target variable in test

y_test.value_counts(normalize=True)*100

class
1.0      65.0
0.0      35.0
dtype: float64

In [21]:
# Angle Encoding

num_qubits = n_dim

dev = qml.device('default.qubit', wires = num_qubits)

@qml.qnode(dev)
def circuit(parameters, data):
    for i in range(num_qubits):
        qml.Hadamard(wires = i)
    
    AngleEmbedding(features = data, wires = range(num_qubits), rotation = 'Y')
    
    qml.StronglyEntanglingLayers(weights = parameters, wires = range(num_qubits))
    
    return qml.expval(qml.PauliZ(0))

In [22]:
num_layers = 5
weights_init = 0.01 * np.random.randn(num_layers, num_qubits, 3, requires_grad=True)
bias_init = np.array(0.0, requires_grad=True)

print(weights_init, bias_init)

[[[ 0.00496714 -0.00138264  0.00647689]
  [ 0.0152303  -0.00234153 -0.00234137]]

 [[ 0.01579213  0.00767435 -0.00469474]
  [ 0.0054256  -0.00463418 -0.0046573 ]]

 [[ 0.00241962 -0.0191328  -0.01724918]
  [-0.00562288 -0.01012831  0.00314247]]

 [[-0.00908024 -0.01412304  0.01465649]
  [-0.00225776  0.00067528 -0.01424748]]

 [[-0.00544383  0.00110923 -0.01150994]
  [ 0.00375698 -0.00600639 -0.00291694]]] 0.0


In [23]:
circuit(weights_init, data[0])

tensor(-0.01754693, requires_grad=True)

In [24]:
def variational_classifier(weights, bias, x):
    return circuit(weights, x) + bias

In [25]:
def square_loss(labels, predictions):
    loss = 0
    for l, p in zip(labels, predictions):
        loss = loss + (l - p) ** 2

    loss = loss / len(labels)
    return loss

In [26]:
def accuracy(labels, predictions):

    loss = 0
    for l, p in zip(labels, predictions):
        if abs(l - p) < 1e-5:
            loss = loss + 1
    loss = loss / len(labels)

    return loss

In [27]:
def cost(weights, bias, X, Y):
    predictions = [variational_classifier(weights, bias, x) for x in X]
    return square_loss(Y, predictions)

In [28]:
Y = np.array(y_train.values[:,0] * 2 - np.ones(len(y_train.values[:,0])), requires_grad = False)  # shift label from {0, 1} to {-1, 1}
X = np.array(data, requires_grad=False)

for i in range(5):
    print("X = {}, Y = {: d}".format(list(X[i]), int(Y[i])))

X = [tensor(3.61802517, requires_grad=False), tensor(0.0623883, requires_grad=False)], Y =  1
X = [tensor(-0.26747551, requires_grad=False), tensor(0.20654232, requires_grad=False)], Y = -1
X = [tensor(3.17667509, requires_grad=False), tensor(0.03027097, requires_grad=False)], Y =  1
X = [tensor(-0.3885942, requires_grad=False), tensor(2.05312899, requires_grad=False)], Y = -1
X = [tensor(-0.25937949, requires_grad=False), tensor(-0.23003446, requires_grad=False)], Y =  1


In [29]:
opt = AdamOptimizer(stepsize=0.1, beta1=0.9, beta2=0.99, eps=1e-08)
batch_size = 10

In [30]:
%%time
weights = weights_init
bias = bias_init

wbest = 0
bbest = 0
abest = 0

for it in range(100):

    # weights update by one optimizer step

    batch_index = np.random.randint(0, len(X), (batch_size,))
    X_batch = X[batch_index]
    Y_batch = Y[batch_index]
    weights, bias, _, _ = opt.step(cost, weights, bias, X_batch, Y_batch)

    # Compute the accuracy
    predictions = [np.sign(variational_classifier(weights, bias, x)) for x in X]
    
    if accuracy(Y, predictions) > abest:
        wbest = weights
        bbest = bias
        abest = accuracy(Y, predictions)
        print('New best')

    acc = accuracy(Y, predictions)

    print(
        "Iter: {:5d} | Cost: {:0.7f} | Accuracy: {:0.7f} ".format(
            it + 1, cost(weights, bias, X, Y), acc
        )
    )

New best
Iter:     1 | Cost: 1.1282195 | Accuracy: 0.4812500 
New best
Iter:     2 | Cost: 1.5428355 | Accuracy: 0.5000000 
Iter:     3 | Cost: 1.1904969 | Accuracy: 0.4937500 
Iter:     4 | Cost: 1.1243809 | Accuracy: 0.4937500 
Iter:     5 | Cost: 1.0026023 | Accuracy: 0.4937500 
New best
Iter:     6 | Cost: 0.9586138 | Accuracy: 0.5812500 
Iter:     7 | Cost: 1.0152774 | Accuracy: 0.5750000 
Iter:     8 | Cost: 1.0102335 | Accuracy: 0.5750000 
Iter:     9 | Cost: 1.0049940 | Accuracy: 0.5750000 
Iter:    10 | Cost: 1.0062718 | Accuracy: 0.5750000 
Iter:    11 | Cost: 1.0148560 | Accuracy: 0.5812500 
Iter:    12 | Cost: 1.0065098 | Accuracy: 0.5812500 
New best
Iter:    13 | Cost: 0.9646521 | Accuracy: 0.6000000 
New best
Iter:    14 | Cost: 0.9429501 | Accuracy: 0.6062500 
Iter:    15 | Cost: 0.9335975 | Accuracy: 0.6000000 
Iter:    16 | Cost: 0.9346379 | Accuracy: 0.5625000 
Iter:    17 | Cost: 0.9315534 | Accuracy: 0.5437500 
Iter:    18 | Cost: 0.9325291 | Accuracy: 0.5375000 
I

In [31]:
Yte = np.array(y_test.values[:,0] * 2 - np.ones(len(y_test.values[:,0])), requires_grad = False)
Xte = np.array(normalize(x_test), requires_grad=False)

In [32]:
predictions = [np.sign(variational_classifier(wbest, bbest, x)) for x in Xte]
pred = [np.sign(variational_classifier(wbest, bbest, x)) for x in X]
acc = accuracy(Yte, predictions)

print(f'Cost: {cost(wbest, bbest, Xte, Yte)}, Accuracy: {np.round(acc, 2) * 100}%')

Cost: 0.9417743846469321, Accuracy: 45.0%


In [33]:
pd.DataFrame((predictions, Yte), ('Predictions', 'Test')).T

,Predictions,Test
0,-1.0,1.0
1,1.0,1.0
2,-1.0,1.0
3,-1.0,-1.0
4,-1.0,1.0
5,-1.0,-1.0
6,-1.0,1.0
7,-1.0,1.0
8,-1.0,1.0
9,-1.0,1.0


In [34]:
# Print the classification report and important metrics

print(metrics.classification_report(predictions,Yte))
print(metrics.precision_score(predictions,Yte))
print(metrics.recall_score(predictions,Yte))
print(metrics.f1_score(predictions,Yte))
print(metrics.balanced_accuracy_score(predictions,Yte))

              precision    recall  f1-score   support

        -1.0       0.93      0.38      0.54        34
         1.0       0.19      0.83      0.31         6

    accuracy                           0.45        40
   macro avg       0.56      0.61      0.43        40
weighted avg       0.82      0.45      0.51        40

0.19230769230769232
0.8333333333333334
0.3125
0.607843137254902
